In [ ]:
# Mount Google Drive to access data
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Подключение к Google drive

In [ ]:
import os
import zipfile

# Extract the video files from the zip file
with zipfile.ZipFile('/content/drive/MyDrive/videos/vid.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/videos')


Загрузка файла

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# Define the network architecture
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])


Самоопределение


In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


Компиляция

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create the data generator
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)


Инструменты для обучения

In [ ]:
# Load the training data
train_generator = train_datagen.flow_from_directory('/content/videos',
                                                    target_size=(256, 256),
                                                    batch_size=32,
                                                    class_mode='binary')


Found 0 images belonging to 18 classes.


Загрузка Видео

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

# download dataset
!wget https://storage.googleapis.com/tf-datasets/titanic/train.csv

# load dataset
dataset = pd.read_csv('train.csv')

# split dataset into training and validation sets
train_dataset = dataset.sample(frac=0.8, random_state=0)
validation_dataset = dataset.drop(train_dataset.index)

# define feature columns
feature_columns = []
for header in ['age', 'n_siblings_spouses', 'parch', 'fare']:
  feature_columns.append(tf.feature_column.numeric_column(header))
age = tf.feature_column.numeric_column("age")
sibsp = tf.feature_column.numeric_column("n_siblings_spouses")
parch = tf.feature_column.numeric_column("parch")
fare = tf.feature_column.numeric_column("fare")

# define input functions
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

train_input_fn = make_input_fn(train_dataset, train_dataset.pop('survived'))
validation_input_fn = make_input_fn(validation_dataset, validation_dataset.pop('survived'), num_epochs=1, shuffle=False)

# create the model
model = tf.keras.Sequential([
  tf.keras.layers.DenseFeatures(feature_columns=feature_columns),
  tf.keras.layers.Dense(units=128, activation='relu'),
  tf.keras.layers.Dense(units=128, activation='relu'),
  tf.keras.layers.Dense(units=1, activation='sigmoid')
])

# compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# train the model
history = model.fit(train_input_fn(), epochs=10, validation_data=validation_input_fn())



--2023-03-04 08:41:05--  https://storage.googleapis.com/tf-datasets/titanic/train.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.23.128, 64.233.189.128, 108.177.97.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.23.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30874 (30K) [application/octet-stream]
Saving to: ‘train.csv’

train.csv           100%[===================>]  30.15K  --.-KB/s    in 0s      

2023-03-04 08:41:06 (68.6 MB/s) - ‘train.csv’ saved [30874/30874]

Epoch 1/10


158/160 [============================>.] - ETA: 0s - loss: 0.6918 - accuracy: 0.6659

160/160 [==============================] - 10s 22ms/step - loss: 0.6909 - accuracy: 0.6659 - val_loss: 0.6381 - val_accuracy: 0.7200
Epoch 2/10
160/160 [==============================] - 2s 11ms/step - loss: 0.6556 - accuracy: 0.6743 - val_loss: 0.8031 - val_accuracy: 0.5520
Epoch 3/10
160/160 [==============================] - 1s 3ms/step - loss: 0.6514 - accuracy: 0.6797 - val_loss: 0.6819 - val_accuracy: 0.6640
Epoch 4/10
160/160 [==============================] - 1s 3ms/step - loss: 0.6129 - accuracy: 0.6863 - val_loss: 0.6655 - val_accuracy: 0.6160
Epoch 5/10
160/160 [==============================] - 1s 3ms/step - loss: 0.5862 - accuracy: 0.6954 - val_loss: 0.6234 - val_accuracy: 0.7280
Epoch 6/10
160/160 [==============================] - 1s 3ms/step - loss: 0.6019 - accuracy: 0.6946 - val_loss: 0.6073 - val_accuracy: 0.7520
Epoch 7/10
160/160 [==============================] - 1s 3ms/step - loss: 0.5887 - accuracy: 0.6984 - val_loss: 0.5985 - val_accuracy: 0.7440
Epoch 8/10
160

Тренировка модели

In [ ]:
# Save the model
model.save('/content/drive/MyDrive/code_recognition_model')


Сохранение

In [ ]:
from flask import Flask, request, jsonify
import tensorflow as tf
import numpy as np

# Define your neural network model here
def predict(input_data):
    # ...
    return output_data

app = Flask(__name__)

@app.route('/')
def index():
    # Serve the HTML file with the input and output windows
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict_handler():
    # Parse input data from the request
    input_data = request.json['input_data']

    # Pass input data to the neural network model
    output_data = predict(input_data)

    # Return output data as a response
    response_data = {'output_data': output_data}
    return jsonify(response_data)

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
